Set up Google Colab: Open Colab, ensure you have GPU access selected (Runtime > Change runtime type > T4 GPU). This is crucial for training later.


Create GitHub Repository: Go to GitHub and create a new repository. This will be used for version control and later for deploying your Streamlit app.



Install Libraries: In your Colab notebook, install all necessary packages.

Python

!pip install datasets transformers torch streamlit gradio

In [1]:
!pip install datasets transformers torch streamlit gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 115.1 MB/s eta 0:00:00


Load SQuAD (Q1.1): Use the datasets library to load the SQuAD dataset

In [2]:
from datasets import load_dataset
squad = load_dataset("squad")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Subset Data (Q1.2): Create the smaller training (3000) and validation (500) sets as specified in the assignment.

In [4]:
train_dataset = squad['train'].shuffle(seed=42).select(range(3000))
validation_dataset = squad['validation'].shuffle(seed=42).select(range(500))

Explore Q&A (Q1.3): Print a few examples to understand the data structure (id, title, context, question, answers).

In [5]:
print("--- Sample 1 ---")
print(f"Context: {train_dataset[0]['context']}")
print(f"Question: {train_dataset[0]['question']}")
print(f"Answer: {train_dataset[0]['answers']}")

--- Sample 1 ---
Context: The Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery.
Question: What percentage of Egyptians polled support death penalty for those leaving Islam?
Answer: {'text': ['84%'], 'answer_start': [468]}


Analyze Statistics (Q1.4): Calculate the average lengths of contexts and questions. This helps in setting tokenization parameters later

In [6]:
import pandas as pd

# Example for context length
context_lengths = [len(example['context']) for example in train_dataset]
print(f"Average Context Length: {pd.Series(context_lengths).mean()}")

# Example for question length
question_lengths = [len(example['question']) for example in train_dataset]
print(f"Average Question Length: {pd.Series(question_lengths).mean()}")

Average Context Length: 760.623
Average Question Length: 58.81166666666667
